In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 18.1 MB/s 
     |████████████████████████████████| 6.6 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 61.1 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
from collections import defaultdict
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelForCausalLM.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
#process mortgage dataset
racenames = {}
names = pd.read_excel('/content/drive/MyDrive/firstnames.xlsx', sheet_name=1)[:-1]
for race in names.columns[2:]:
    raceidx = names[names.drop(columns=['firstname', 'obs']).idxmax(axis=1)==race]['obs'].sort_values(ascending=False)[:200].index
    racenames[race[3:]]=names.loc[raceidx]['firstname'].values

#process social security data
gendernames=defaultdict(lambda : defaultdict(int))
for year in range(1920,2022):
    with open(f'/content/drive/MyDrive/names/yob{year}.txt') as f:
        for line in f:
            name,gender,number=line.split(',')
            gendernames[name][gender]+=int(number)

#split mortgage data using social security data
namedict = defaultdict(lambda: defaultdict(list))
for race in list(racenames.keys())[:-2]:
    for name in racenames[race]:
        name = name.capitalize()
        if gendernames[name]['F'] > gendernames[name]['M']:
            namedict[race]['female'].append(name)
        elif gendernames[name]['F'] < gendernames[name]['M']:
            namedict[race]['male'].append(name)
        else:
            #names in mortgage data not in social security data
            print(name)

Venkata
Hua
Pankaj
Byung
Rong
Qiang


In [ ]:
#print number of names per class
for race in namedict.keys():
    for gender in namedict[race].keys():
        print( race, gender, len(namedict[race][gender]))

hispanic female 85
hispanic male 115
white male 92
white female 108
black male 31
black female 39
api male 132
api female 62


In [ ]:
#process mortgage data without selecting 200 most popular
racenames2 = {}
names = pd.read_excel('/content/drive/MyDrive/firstnames.xlsx', sheet_name=1)[:-1]
for race in names.columns[2:]:
    raceidx = names[names.drop(columns=['firstname', 'obs']).idxmax(axis=1)==race]['obs'].sort_values(ascending=False).index
    racenames2[race[3:]]=names.loc[raceidx]['firstname'].values

#process social security data
gendernames=defaultdict(lambda : defaultdict(int))
for year in range(1920,2022):
    with open(f'/content/drive/MyDrive/names/yob{year}.txt') as f:
        for line in f:
            name,gender,number=line.split(',')
            gendernames[name][gender]+=int(number)

#filter for only names tokenized to 1 token en split using social security data
namedict2 = defaultdict(lambda: defaultdict(list))
for race in list(racenames2.keys())[:-2]:
    for name in racenames2[race]:
        name = name.capitalize()
        if len(tokenizer(name)['input_ids']) == 1:
            if gendernames[name]['F'] > gendernames[name]['M']:
                namedict2[race]['female'].append(name)
            elif gendernames[name]['F'] < gendernames[name]['M']:
                namedict2[race]['male'].append(name)

#print number of names per class
d = defaultdict(lambda:defaultdict(int))
for race in namedict2.keys():
    for gender in namedict2[race].keys():
        for name in namedict2[race][gender]:
            d[race][gender]+=1
d

defaultdict(<function __main__.<lambda>>,
            {'api': defaultdict(int, {'female': 27, 'male': 45}),
             'black': defaultdict(int, {'male': 3}),
             'hispanic': defaultdict(int, {'female': 3, 'male': 8}),
             'white': defaultdict(int, {'female': 58, 'male': 194})})